# 神经网络的结构 笔记
教程视频链接：https://www.bilibili.com/video/BV1hE411t7RN

这篇笔记对应视频合集中的
- 神经网络-卷积层
- 神经网络-最大池化的使用
- 神经网络-非线性激活
- 神经网络-线性层及其他层介绍

本节介绍了运用torch.nn类构建神经网络的流程，对于一些没有介绍到的层，可以参考官方文档。

相关的官方文档：https://pytorch.org/docs/stable/nn.html#module-torch.nn

卷积、池化、非线性激活是处理数据的方法，在神经网络构建过程中，卷积 -> 池化 -> 非线性激活，三者一般配套使用，下面分别介绍这三种方法。
## 1.卷积层（Convolution Layers）
模板包括一维卷积、二维卷积等等，处理图片一般使用二维卷积，这里以此为例展开

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d


In [1]:
#创建dataset和dataloader
import torch
import torchvision
from torch.utils.data import DataLoader

dataset = torchvision.datasets.CIFAR10(root='../dataset', train=False,download=False, transform=torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset=dataset, batch_size=64)

In [2]:
#导入二维卷积模板类
from torch.nn.modules import Conv2d

#创建神经网络模板子类
class Tudui(torch.nn.Module):
    #类初始化
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0)
        '''
        卷积操作对象。常用参数：
        stride -> 卷积的步长
        padding -> 外层填充量
        kernel_size -> 卷积核的大小
        in_channels -> 输入数据通道数，黑白为1，彩色为3
        out_channels -> 输出的通道数（这个概念已经脱离图像本身了）
        '''
    
    #forward函数，
    def forward(self, x):
        x = self.conv1(x)
        return x
    

In [ ]:
#创建一个神经网络对象
tudui = Tudui()
print(tudui)

#打印卷积核kernel，由于没有手动设置，在实例化过程中torch自动生成了卷积核
print(tudui.conv1.weight.shape)
print(tudui.conv1.weight)

In [ ]:
for data in dataloader:
    imgs, targets = data#tensor类型的图片
    output = tudui(imgs)
    print(imgs.shape, output.shape)#输出torch.Size([64, 3, 32, 32]) torch.Size([64, 6, 30, 30])，最后一个batch可能不是64张

在这个例子中，神经网络的input为tensor格式，其shape为
$$inshape = (B, C_{in}, H, W)$$
其中：`B`为batch_size，`C`为in_channels，`H`和`W`为图像的长宽。

对应的output同样为tensor格式，其shape为
$$outshape = (B, C_{out}, H', W')$$
其中`C`为out_channels，由卷积核kernel的数量决定， `H'`和`W'`由图片相关参数和卷积操作相关参数决定，即
$$H' = \left\lfloor\frac{H + 2 \times \text{padding} - K}{\text{stride}}\right\rfloor + 1$$
$$W' = \left\lfloor\frac{W + 2 \times \text{padding} - K}{\text{stride}}\right\rfloor + 1$$
对于卷积、池化等概念的形象理解：https://www.bilibili.com/video/BV1nk4y1271L



In [5]:
#用tensorboard展示经过卷积操作的图片
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("..\logs")

step = 0
for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    #torch.Size([64, 3, 32, 32]) 
    writer.add_images(tag='input', img_tensor=imgs, global_step=step)#注意！这里是add_images，不是add_image！
    #torch.Size([64, 6, 30, 30])，6通道无法显示，需要reshape（强制转换）
    output = torch.reshape(output, (-1, 3, 30, 30))#把多的通道处理为更多的batch，batch_size填-1会自动计算
    writer.add_images(tag='output', img_tensor=output, global_step=step)#注意！这里是add_images，不是add_image！
    step += 1

writer.close()

## 2.池化层（Pooling Layers）
最大池化可以降低样本的维度，同时保留特征信息，进行该操作可以提升模型的鲁棒性，尤其是面对扭曲和旋转的情况下。此外，池化操作可以减小进入神经网络的数据量，提升数据的信息密度，加快模型训练的速度。pytorch提供了池化层的模板，可以实例化添加到自定义神经网络中。这里用二维最大池化（MaxPool2D）举例，最大池化又被称为下采样（subsampling）。

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d

In [6]:
class Tudui2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.maxpool = torch.nn.MaxPool2d(kernel_size=3, ceil_mode=True)
        '''
        创建最大池化层。常用参数：
        dilation -> 空洞池化核（一般不设置）
        ceil_mode -> ceiling/floor 指向上或向下取整，此处指池化核覆盖虚空时是否保留
        '''
    def forward(self, x):
        return self.maxpool(x)
tudui = Tudui2()
    
#显示输出的结果，效果类似有损压缩
writer = SummaryWriter('../logs')
step = 0
for data in dataloader:
    imgs, tags = data
    output = tudui(imgs)
    writer.add_images(tag='inputs', global_step=step, img_tensor=imgs)
    writer.add_images(tag='outputs', global_step=step, img_tensor=output)
    step +=1
    

池化操作输入与输出的tensor数据的shape与卷积操作中类似，不同的是池化中的输出数据通道数$C_{out}$不由kernel数量决定（池化操作不存在多个kernel），而是与输入数据通道数$C_{in}$的值相同。

In [7]:
#在自定义数据上操作时，存在一个数据类型的问题，此时给tensor数据类型加上`dtype=torch.float32`即可解决问题：

input = torch.tensor([[[1, 2, 3, 4, 5],
                      [6, 7, 8, 9, 0],
                      [2, 5, 1, 6, 3],
                      [3, 5, 1, 2, 1],
                      [7, 3, 5, 3, 6]]])#有的版本的pytorch这样写会报错

input = torch.tensor([[[1, 2, 3, 4, 5],
                      [6, 7, 8, 9, 0],
                      [2, 5, 1, 6, 3],
                      [3, 5, 1, 2, 1],
                      [7, 3, 5, 3, 6]]], dtype=torch.float32)#如果报错加上这行即可
tudui(input)

tensor([[[8., 9.],
         [7., 6.]]])

## 3.非线性激活（Non-linear Activations）
用非线性的激活函数可以给神经网络添加非线性成分，赋予神经网络解决非线性问题的能力。如果没有非线性激活函数，无论神经网络有多少隐含层，都会等效于一个没有隐含层的线性变换。

对该问题的说明：https://www.bilibili.com/video/BV1Yh4y1c7sT

pytorch提供了对应多种非线性激活函数的非线性激活层模板，可以直接将非线性激活层实例化，添加到神经网络模板子类中。

官方文档：https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity

在各种非线性激活函数中，较为常用的有ReLU函数（Rectified Linear Unit Function）：
$$\text{ReLU}(x) = \text{max}(0, x)$$
Sigmoid激活函数（Sigmoid activation function）：
$$\text{Sigmoid}(x) = \frac{1}{1 + e^{-x}}$$
下面以ReLU函数为例，说明添加非线性激活层的方法

In [9]:
import torch

class Tudui3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.relu1 = torch.nn.ReLU(inplace=False)
        '''
        创建非线性激活层。常用参数：
        inplace -> 是否替换原有变量，设置为False（默认值）可以保留原有数据，更方便
        '''
        self.sigmoid = torch.nn.Sigmoid()#sigmoid函数非线性激活，会让图片处理效果更明显
    def forward(self, x):
        x = self.relu1(x)
        x = self.sigmoid(x)
        return x
tudui = Tudui3()

#tensorboard展示输出结果
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('../logs')
step = 0
for data in dataloader:
    imgs, tags = data
    output = tudui(imgs)
    writer.add_images(tag='inputs', global_step=step, img_tensor=imgs)
    writer.add_images(tag='outputs', global_step=step, img_tensor=output)
    step +=1
    
writer.close()

## 4.归一化层（Normalization Layers）
处理输入的tensor数据，使之在不丧失数据原有特征的情况下，均值变为0，方差变为1。这样的处理可以加快神经网络的训练速度。

关于归一化层参考视频：https://www.bilibili.com/video/BV1f8411w75v

pytorch提供了不同归一化层的模板，这里用`nn.BatchNorm2d`处理图片数据举例。

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d

In [ ]:
import torch

class Pies(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.norm = torch.nn.BatchNorm2d(num_features=3)
        '''
        创建归一化层，常用参数：
        num_features -> 特征数量，可以理解为输入数据通道数量
        '''
    def forward(self, x):
        return self.norm(x)
pies = Pies()

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('../logs')
step = 0
for data in dataloader:
    imgs, tags = data
    output = pies(imgs)
    writer.add_images(tag='inputs', global_step=step, img_tensor=imgs)
    writer.add_images(tag='outputs', global_step=step, img_tensor=output)
    step +=1
    
writer.close()

## 5.线性层（Linear Layers）
线性层对一维数据进行线性变换，一般用于展示预测结果（通过线性变换将vector变为scaler）。pytorch为线性层提供了多种模板，这里以一般形式`nn.Linear`模板为例。该过程的公式为
$$y = xA^T + b$$
其中`A`为weight matrix`B`为bias vector二者均是自动初始化，且在训练过程中逐渐学习到的

关于线性层参考视频：https://www.bilibili.com/video/BV1o94y1Y7G9

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.Linear

In [ ]:
import torch

class Tudui4(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(in_features=196608, out_features=1)#此处的in_features为数据集中数据展平后的维数
        '''
        创建线性层。常用参数：
        in_features -> 输入数据的维数
        out_fetures -> 输出数据的维数
        '''
    def forward(self, x):
        return self.linear(x)
tudui = Tudui4()

for data in dataloader:
    imgs, targets = data
    input = torch.flatten(imgs)#torch.flatten可以将多维度tensor数据转换为vector数据
    if input.shape[0] == 196608:
        output = tudui(input)
    print(input.shape, output.shape, output)
